# [![](https://img.shields.io/badge/Video-H∆∞·ªõng%20d·∫´n-ff0000)](https://youtu.be/798qchqWty4) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-2.0-0075ff)](https://stablediffusion.vn/update/) [![](https://img.shields.io/badge/All%20Tools-sdvn.me-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/badge/SDVN-Library-green)](https://bit.ly/sdvn-lib) [![](https://img.shields.io/badge/Kho√°%20h·ªçc-SD%20Flux-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---
üß® `Train ƒë∆∞·ª£c ·ªü L4 (Y√™u c·∫ßu b·∫≠t low VRAM - Batch_size 1) v√† A100`

üî• `C·∫≠p nh·∫≠t phi√™n b·∫£n v2, h·ªó tr·ª£ train nhi·ªÅu lora trong m·ªôt setting, th√™m ch·∫ø ƒë·ªô s·ª≠ d·ª•ng v·ªõi giao di·ªán UI`

#‚òïÔ∏è 1.C√†i ƒë·∫∑t

In [ ]:
#@title ‚ú® C√†i ƒë·∫∑t UI

# @markdown üü° `L·ª±a ch·ªçn Gradio n·∫øu mu·ªën train tr·ª±c ti·∫øp v·ªõi giao di·ªán UI (S·ª≠ d·ª•ng link gradio) `
%cd /content
from google.colab import drive
drive.mount('/content/drive')

UI = 'Colab' # @param ['Gradio','Colab']

root_dir = "/content"
repo_dir = f"{root_dir}/SDVN-training-colab-flux"
toolkit_dir = f"{repo_dir}/ai-toolkit"
config_dir = f"{repo_dir}/Config"

%cd {root_dir}
import os
if not os.path.exists(repo_dir):
  !git clone  --recurse-submodules https://github.com/StableDiffusionVN/SDVN-training-colab-flux {repo_dir}
  %env TF_CPP_MIN_LOG_LEVEL=1
  %env LD_PRELOAD={repo_dir}/libtcmalloc_minimal.so.4
%cd {toolkit_dir}
!pip install -q -r requirements.txt

%run {repo_dir}/TrainScript.ipynb

if UI == "Gradio":
  !rm -r {toolkit_dir}/output
  !ln -s /content/drive/MyDrive/SD-Data/Lora {toolkit_dir}/output
  !python {toolkit_dir}/flux_train_ui.py


#‚è≥ 2. C·∫•u h√¨nh train

In [ ]:
#@title ‚ú® 2.1 X·ª≠ l√Ω d·ªØ li·ªáu
# @markdown üü° `Train nhi·ªÅu lora b·∫±ng c√°ch nh·∫≠p nhi·ªÅu th∆∞ m·ª•c, c√°ch nhau b·∫±ng d·∫•u ph·∫©y (,)`
TrainFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
# @markdown üü° `ƒêi·ªÅu ch·ªânh m·ª©c ƒë·ªô d√†i caption t·ª± ƒë·ªông`
Caption = 'Florence' # @param ['None','Blip','Florence']
Caption_Length = "Medium" # @param ['Short','Medium','Long']
# @markdown üü° `Th√™m caption tu·ª≥ ch·ªçn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}
extension =".txt"
train_folder_list = TrainFolder.split(",")

Cap_prompt = {
    'Short':['<CAPTION>',10,30],
    'Medium':['<DETAILED_CAPTION>',10,100],
    'Long':['<MORE_DETAILED_CAPTION>',10,150]
}

for folder in train_folder_list:
  if DataClean == True :
    %cd /content
    clean_directory(folder)
  if Caption != "None":
    caption_dir(folder,Cap_prompt[Caption_Length][0])
  if AddFolderName:
    add_forder_name(folder)
  if Custom_Caption != "":
    process_dir(folder, Custom_Caption, Append, Remove_Caption)

In [ ]:
#@title ‚öôÔ∏è 2.2 C√†i ƒë·∫∑t train
Low_VRAM = False #@param {type:'boolean'}
Lora_name = "lora_name" # @param {"type":"string","placeholder":"T√™n lora"}
OutputFolder = "/content/drive/MyDrive/SD-Data/Lora" #@param {type:'string'}
# @markdown üü° `ƒêi·ªÅu ch·ªânh steps theo m·ª©c ƒë·ªô train mong mu·ªën, c√°c th∆∞ m·ª•c ƒë·∫∑t t√™n theo c√∫ ph√°p {steps}_{name} s·∫Ω t·ª± nh·∫≠n steps ri√™ng theo t√™n`
Steps = 1000 #@param {type:'number'}
Save_steps = 250 #@param {type:'number'}
Batch_size = 4 #@param {type:'number'}
# @markdown üü° `C√°c th√¥ng s·ªë n√™n ƒë·ªÉ theo m·∫∑c ƒë·ªãnh`
Lr = 1e-4 #@param {type:'number'}
Dim = 16 #@param {type:'number'}
Alpha = 16 #@param {type:'number'}
Resolution = "[ 512, 768, 1024 ]" #@param {type:'string'}


folder_train = []
for folder in train_folder_list:
  if len(get_supported_images(folder)) > 0:
    folder_dic = {
    "path": folder,
    "steps": get_steps(folder)[0],
    "name": get_steps(folder)[1].replace(" ", "_")
    }
    folder_train.append(folder_dic)
  else:
    print(f"Th∆∞ m·ª•c [ {folder} ] c√≥ th·ªÉ kh√¥ng ch·ª©a ·∫£nh ƒë∆∞·ª£c h·ªó tr·ª£, h√£y ki·ªÉm tra l·∫°i (.png, .jpg, .jpeg, .webp, .bmp, .JPG, .JPEG, .PNG)")

for folder in folder_train:
  file_path = f'{config_dir}/config_{folder["name"]}.yaml'
  !cp "{config_dir}/config.yaml" "{file_path}"
  if AddFolderName:
    process_dir(folder["path"], folder["name"], False, False)
  config = {
      'data': [r'folder_path:.*',f'folder_path: "{folder["path"]}"'],
      'name': [r'name:.*',f'name: "{Lora_name if len(folder_train) == 1 else folder["name"]}"'],
      'output': [r'training_folder:.*',f'training_folder: "{OutputFolder}"'],
      'steps': [r' steps:.*',f' steps: {folder["steps"]}'],
      'save_steps': [r'save_every:.*',f'save_every: {Save_steps}'],
      'sample_every': [r'sample_every:.*',f'sample_every: {Save_steps}'],
      'resolution': [r'resolution:.*',f'resolution: {Resolution}'],
      'batch': [r'batch_size:.*',f'batch_size: {Batch_size}'],
      'dim': [r'linear:.*',f'linear: {Dim}'],
      'alpha': [r'linear_alpha:.*',f'linear_alpha: {Alpha}'],
      'lr': [r'lr:.*',f'lr: {Lr}'],
      'prompt': [r'prompts:[\s\S]*?neg:',f'prompts:\n          - "{random_sample(folder["path"])}"\n        neg:'],
      'low_vram': [r'low_vram:.*',f'low_vram: {"true" if Low_VRAM == True else "false"}'],

  }
  for i in config:
    replace(config[i][0],config[i][1])

  print('=====================')
  print(f'Th∆∞ m·ª•c train: {folder["path"]}')
  print(f'  S·ªë l∆∞·ª£ng ·∫£nh: {len(get_supported_images(folder["path"]))}')
  print(f'  S·ªë Steps: {folder["steps"]}')
  print(f'  Output: {OutputFolder}/{folder["name"]}')
  print('=====================')


#3. üß® Train

In [ ]:
#@title ‚è≥ Run Lora Train
train()